In [1]:
import json

train_file = "/kaggle/input/vietnamese-squad/train-v2.0-translated.json"
dev_file = "/kaggle/input/vietnamese-squad/dev-v2.0-translated.json"
with open(train_file, "r", encoding="utf-8") as f:
    train_data = json.load(f)

with open(dev_file, "r", encoding="utf-8") as f:
    dev_data = json.load(f)

In [2]:
train_data[0]

['Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên ở Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn nhỏ, và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ R & B Destiny\'s Child. Được quản lý bởi cha cô, Mathew Knowles, nhóm đã trở thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạn của họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love (2003), giúp cô trở thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải Grammy và có đĩa đơn quán quân Billboard Hot 100 "Crazy in Love" và "Baby Boy".',
 'Beyonce bắt đầu nổi tiếng từ khi nào?',
 'Vào cuối những năm 1990']

In [3]:
def change_format(data):
    json_data = []
    for d in data:
        sample = {"context": d[0],
                "question": d[1],
                "ans": d[2]}

        json_data.append(sample)
    return json_data

train_data = change_format(train_data)
dev_data = change_format(dev_data)

In [4]:
train_data[0]

{'context': 'Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên ở Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn nhỏ, và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ R & B Destiny\'s Child. Được quản lý bởi cha cô, Mathew Knowles, nhóm đã trở thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạn của họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love (2003), giúp cô trở thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải Grammy và có đĩa đơn quán quân Billboard Hot 100 "Crazy in Love" và "Baby Boy".',
 'question': 'Beyonce bắt đầu nổi tiếng từ khi nào?',
 'ans': 'Vào cuối những năm 1990'}

In [5]:
import torch
def create_segment_id(tokenized_input):
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = tokenized_input['input_ids'].index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(tokenized_input['input_ids']) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(tokenized_input['input_ids'])
    
    return segment_ids

In [6]:
import re
def find_start_end_char(ques, answer, context):
    match = re.search(re.escape(answer), context, re.IGNORECASE)
    if match:
        start_char = match.start()
        end_char = match.end()
    else:
        start_char, end_char = -1, -1
    return start_char, end_char 

In [7]:
import os
import wandb

with open('/kaggle/input/wandb-key/wandb_key.txt', 'r') as f:
    os.environ['WANDB_API_KEY'] = f.read().strip()
    print(f.read().strip())

# Initialize wandb without interactive login
wandb.login(key=os.environ['WANDB_API_KEY'])

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


wandb: Currently logged in as: ntkhang2003 (ntkhang2003-university-of-information-technology-vnuhcm). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [8]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import BertForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForQuestionAnswering.from_pretrained("bert-base-multilingual-cased")

def preprocess_function(example):
    inputs = tokenizer(
        example["question"], 
        example["context"], 
        max_length=128, 
        padding="max_length",
        truncation=True,
        return_offsets_mapping=True
    )
    
    start_char, end_char = find_start_end_char(example['question'], example["ans"], example["context"])
    if start_char == -1 or end_char == -1:
        inputs["start_positions"], inputs["end_positions"] = -1, -1
    else:     
        # Convert character positions to token positions
        offsets = inputs["offset_mapping"]
        inputs["start_positions"], inputs["end_positions"] = 0, 0
        for idx, (start, end) in enumerate(offsets):
            if start == start_char:
                inputs["start_positions"] = idx
            if end == end_char:
                inputs["end_positions"] = idx
                break
        inputs.pop("offset_mapping")  # Remove offset_mapping after use
    
    return inputs

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from tqdm import tqdm 
from datasets import Dataset

def tokenize_data(squad_formatted_data):
    
    tokenized_dataset = []
    for item in tqdm(squad_formatted_data):
        tokenized_input = preprocess_function(item)
        segment_ids = create_segment_id(tokenized_input)
        tokenized_input["token_type_ids"] = segment_ids
        tokenized_dataset.append(tokenized_input)
    
    return tokenized_dataset

# Tokenize and add custom segment_ids
tokenized_train_data = tokenize_data(train_data)
tokenized_dev_data = tokenize_data(dev_data)

# Convert to Dataset    
tokenized_train_data = Dataset.from_list(tokenized_train_data)
tokenized_dev_data = Dataset.from_list(tokenized_dev_data)

100%|██████████| 11873/11873 [00:12<00:00, 954.18it/s]


## Train model

In [10]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments, BertForQuestionAnswering


# Use the tokenizer you used in preprocess_function
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",           # Evaluate and save at the end of each epoch
    save_strategy="epoch",                 # Save model at each evaluation step
    load_best_model_at_end=True,           # Load the best model at the end of training
    metric_for_best_model="eval_loss",     # Metric to determine the best model (can adjust as needed)
    greater_is_better=False,               # Set to False if lower is better (e.g., for loss)
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=25,
    weight_decay=0.01,
    fp16=True,                             # Use mixed precision for memory efficiency
    gradient_accumulation_steps=4,         # Simulate larger batch size
    save_total_limit=1                     # Keep only the best checkpoint
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_dev_data,  # replace with a validation set if available
    data_collator=data_collator,
)

# Train the model
trainer.train()


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241027_164815-rcur6m3w
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./results
wandb: ⭐️ View project at https://wandb.ai/ntkhang2003-university-of-information-technology-vnuh

Epoch,Training Loss,Validation Loss
0,0.864400,0.718122
1,0.722900,0.687743
2,0.589200,0.713341
4,0.364200,0.871855
5,0.292400,0.982655
6,0.233800,0.995692
8,0.151100,1.218066
9,0.125100,1.264353
10,0.102400,1.379687
12,0.070800,1.486768


TrainOutput(global_step=25450, training_loss=0.18558537560970703, metrics={'train_runtime': 37353.0949, 'train_samples_per_second': 87.221, 'train_steps_per_second': 0.681, 'total_flos': 2.1277342641661133e+17, 'train_loss': 0.18558537560970703, 'epoch': 24.993862018168425})

## Eval model

In [11]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Save the model and tokenizer
model.save_pretrained("./fine_tuned_xlm-roberta")
tokenizer.save_pretrained("./fine_tuned_xlm-roberta")


Evaluation results: {'eval_loss': 0.6877434849739075, 'eval_runtime': 44.2872, 'eval_samples_per_second': 268.091, 'eval_steps_per_second': 8.4, 'epoch': 24.993862018168425}


('./fine_tuned_xlm-roberta/tokenizer_config.json',
 './fine_tuned_xlm-roberta/special_tokens_map.json',
 './fine_tuned_xlm-roberta/vocab.txt',
 './fine_tuned_xlm-roberta/added_tokens.json',
 './fine_tuned_xlm-roberta/tokenizer.json')

## Run inference

In [12]:
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device
model.to(device)

def answer_question(context, question):
    inputs = tokenizer(question, context, return_tensors="pt").to(device)
   
    with torch.no_grad():
        outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits
    start_idx = torch.argmax(start_scores)
    end_idx = torch.argmax(end_scores) + 1

    answer = tokenizer.decode(inputs["input_ids"][0][start_idx:end_idx])
    return answer

In [13]:
for sample in dev_data[:5]:
    
    print("Answer: ", sample['ans'])
    print("Predict answer: ", answer_question(sample['context'], sample['question']))

Answer:  Pháp
Predict answer:  Pháp
Answer:  Thế kỷ 10 và 11
Predict answer:  [CLS] Người Norman ở Normandy khi nào? [SEP] Người Norman ( Norman : Nourmands ; French : Normands ; Latin : Normanni ) là những người trong thế kỷ 10 và 11
Answer:  Đan Mạch, Iceland và Na Uy
Predict answer:  [CLS]
Answer:  Trang chủ
Predict answer:  [CLS]
Answer:  Thế kỉ thứ 10.
Predict answer:  [CLS]
